# Sessie 1 - Introductie

Als een introductiesessie zullen we wat simpele oefeningen maken rond enkele basisstellingen en -begrippen om deze op te frissen en kennis te maken met de notebooks.

Voer de onderstaande code uit om de nodige libraries in te laden.
Je kan natuurlijk op ieder moment zelf libraries importeren om een probleem op te lossen.

In [ ]:
%matplotlib inline

import os
import math # native
import matplotlib.pyplot as plt # te installeren
import numpy as np # te installeren
from scipy import misc # te installeren

def plot_errors(n, err):
    plt.plot(n, err)
    plt.xlabel('n')
    plt.ylabel('error')
    plt.show()

## 1. Algoritme van Horner

Schrijf een programma die een veelterm
$$p(x) = a_0 + a_1x + a_2x^2 + \cdots + a_nx^n$$
evalueert. Gebruik hiervoor het algoritme van Horner (https://en.wikipedia.org/wiki/Horner%27s_method). Dit is het algoritme die het minst aantal bewerkingen gebruikt (in dit geval $2n$).

In [ ]:
def horner(a, x):
    """
    Evalueer een veelterm in punt x. 
    
    Voorbeeld
    ---------
    
    b[0] = a[-1]
    b[1] = a[-2] + b[0]x
    b[2] = a[-3] + b[1]x
    ...
    
    Parameters
    ----------
    a: numpy.array of python list
        Bevat de coefficienten van de veelterm. Opgelet: Deze implementatie vereist dat de coefficienten
        geordend zijn van lage naar hoge order, m.a.w a0, a1, a2, ... , an.
    x: int of float
        Punt waarin de functie geevalueerd moet worden.
    
    Raises
    ------
    None
    
    Returns
    -------
    b: float
        Waarde van veelterm in punt x.
    """
    
    # We beginnen met de coefficient die hoort bij de hoogste orde (dus achteraan de list of numpy array).
    b = a[-1]
    
    # Vervolgens itereren we over de verschillende coefficienten in omgekeerde richting. We starten hier
    # bij len(a) - 2 omdat we a[-1] al gedaan hebben.  
    for n in range(len(a) - 2, -1, -1):
        
        b = a[n] + b*x
        
    return b

In [ ]:
### TEST ###
a = np.array([-1, 2, -6, 2])
x = 3
print(horner(a, x)) #should be equal to 5

## 2. Taylorexpansie

Maak gebruik van de stelling van Taylor om de functie $e^x$ te benaderen.

Afgeleide van $e^x$ = $e^x$.

We kiezen $a = 0$, waardoor $e^0 = 1$.


$$ e^x = f(a) + \frac{f'(a)}{1!}(x-a) + \frac{f''(a)}{2!}(x-a)^2 + \frac{f^{(3)}(a)}{3!}(x-a)^3 + ... $$

$$ = e^0 + \frac{e^0}{1!}(x-0) + \frac{e^0}{2!}(x-0)^2 + \frac{e^0}{3!}(x-0)^3 + ... $$

$$ = 1 + \frac{x}{1!} + \frac{x^2}{2!} + \frac{x^3}{3!} + ... $$

$$ = \sum_{n=0}^{\infty} \frac{x^n}{n!} $$

In [ ]:
def taylor_exp(x, n):
    
    """
    Benader f(x) = e^x tot en met de n-de afgeleide in de Taylorexpansie. Aangezien er geen a meegegeven kan 
    worden als parameter mag u er vanuitgaan dat a = 0. Het gaat hier met andere woorden om een MacLaurin 
    expansie.
    
    Voorbeeld
    ---------
    First order approx.: y = x^0 / 0! + x^1 / 1!
    Second order approx.: y = x^0 / 0! + x^1 / 1! + x^2 / 2!
    ...
    
    Parameters
    ----------
    x: int or float
        Point in which the approximation needs to be evaluated.
    n: int
        Order of the approximation.
    
    Raises
    ------
    None
    
    Returns
    -------
    y: float
        The value corresponding to the approximation evaluated in point x.
    """
    
    y = 0
    
    # The approximation is build up iteratively. Each iteration a the order of the approximation is increased.
    for i in range(n + 1):
        
        y += x ** i / math.factorial(i)
    
    return y

In [ ]:
#TEST
x = 2
n = range(10)
errors = [abs(taylor_exp(x, i) - math.exp(x)) for i in n]
plot_errors(n, errors)

## 3. Gram-Schmidt Orthogonalisatie

Gegeven een basis $V \in \mathbb{R}^{n \times n}$ voor een ruimte $\mathbb{R}^n$, implementeer Gram-Schmidt om een orthogonale basis $Q \in \mathbb{R}^{n \times n}$ te vinden voor diezelfde ruimte.

In [ ]:
def gram_schmidt(V):
    """
    Berekent een orthogonale basis voor dezelfde ruimte gedefinieerd door de basis V. 
    De eerste vector van Q komt overeen met die van V. De nieuwe ruimte wordt iteratief
    opgebouwd. 
    
    Voorbeeld
    ---------
    q[0] = v[0]
    q[1] = v[1] - q'[0]v[1] / q'[0]q[0] * q[0]
    q[2] = v[2] - q'[0]v[2] / q'[0]q[0] * q[0] - q'[1]v[2] / q'[1]q[1] * q[1]
    ...
    
    Parameters
    ----------
    V: list of numpy arrays
        Bevat de oorspronkelijke basis V.
    
    Raises
    ------
    None
    
    Returns
    -------
    Q: list of numpy arrays
        Bevat de nieuwe orthogonale basis Q die dezelfde ruimte definieert als V.
    """
    
    Q = []
    
    # Stap 0: De eerste vector van de nieuwe basis Q is gelijk aan die van V.
    Q_0 = V[0]
    Q.append(Q_0)
    
    # Stap 1 -> ?: Met iedere iteratie wordt er een nieuwe vector gegenereerd van de nieuwe orthogonale basis Q.
    for m in range(1, len(V)):
                
        # tmp houdt het gedeelte dat afgetrokken moet worden van de vector V[m]
        tmp = 0
        for i in range(0, m):
            tmp += np.dot(Q[i], V[m]) * Q[i] / np.dot(Q[i], Q[i])
        
        # Berekenen Q[m].
        Q_m = V[m] - tmp
        
        Q.append(Q_m)
    
    return Q

In [ ]:
# alternatieve implementatie met python list comprehensions
# list comprehensions zullen in de komende WPOs vaker en vaker gebruikt worden
def gram_schmidt_python_listcomprehension(V):
    """
    Bereken een orthogonale basis voor dezelfde ruimte gedefinieerd door de basis V.
    """
    Q = []
    for m in range(len(V)):
        Q.append(V[m] - sum((np.dot(Q[i], V[m]) * Q[i] / np.dot(Q[i], Q[i]) for i in range(m))))
    return(Q)

In [ ]:
#TEST
V = [np.array([4, -3, -2]),  #v1
     np.array([2,  3,  2]),  #v2
     np.array([5,  7,  8])   #v3
    ]
print('<v[0], v[1]> = ' + str(np.dot(V[0], V[1])))
print('<v[0], v[2]> = ' + str(np.dot(V[0], V[2])))
print('<v[1], v[2]> = ' + str(np.dot(V[1], V[2])))

Q = gram_schmidt(V)
print('\n<q[0], q[1]> = ' + str(np.dot(Q[0], Q[1])))
print('<q[0], q[2]> = ' + str(np.dot(Q[0], Q[2])))
print('<q[1], q[2]> = ' + str(np.dot(Q[1], Q[2])))

In [ ]:
#TEST
V = [np.array([4, -3, -2]),  #v1
     np.array([2,  3,  2]),  #v2
     np.array([5,  7,  8])   #v3
    ]
print('<v[0], v[1]> = ' + str(np.dot(V[0], V[1])))
print('<v[0], v[2]> = ' + str(np.dot(V[0], V[2])))
print('<v[1], v[2]> = ' + str(np.dot(V[1], V[2])))

Q = gram_schmidt_python_listcomprehension(V)
print('\n<q[0], q[1]> = ' + str(np.dot(Q[0], Q[1])))
print('<q[0], q[2]> = ' + str(np.dot(Q[0], Q[2])))
print('<q[1], q[2]> = ' + str(np.dot(Q[1], Q[2])))

## 4. Stelling van Rayleigh

Gegeven de matrix $A$
$$A = \begin{pmatrix}
−2 & −2 & 4\\
−4 & 1 & 2\\
2 & 2 & 5
\end{pmatrix}$$
met grootste eigenwaarde $\lambda_1 = 6$

Schrijf een programma dat de bijhorende eigenvector $x_1$ benadert, gebruikmakend van de stelling van Rayleigh.

Extra vraag: geldt de stelling van Rayleigh voor deze matrix gegarandeerd?

In [ ]:
def rayleigh(M, x0, eigval, n):
    
    """
    Gegeven een matrix M met grootste eigenwaarde 'eigval', benader de geassocieerde eigenvector x. (x0.x != 0)
    
    Parameters
    ----------
    M: numpy matrix
        Is de matrix waarvan eigval de grootste eigenwaarde is.
    x0: numpy matrix
        Is de vector vanwaaruit de benadering vertrekt.
    eigval: int
        Is de grootste eigenwaarde van de matrix M.
    n: int
        Is het aantal maal dat we M/eigval met zichzelf vermenigvuldigen. 
        Hoe hoger dit getal, hoe beter de benadering.
    
    Raises
    ------
    None
    
    Returns
    -------
    x1: numpy matrix
        Is de benadering van de eigenvector die hoort bij de grootste eigenwaarde.
    """
    
    # De eigenvector kan benaderd worden door een willekeurige vector x0 (die niet orthogonaal is aan de te 
    # benaderen eigenvector) n keer te vermenigvuldigen met de ratio van de matrix M en de grootste eigenwaarde.
    x = (M / eigval) ** n * x0 
    
    # Resultaat in x is nu c*x1 (zie slide 19)
    #   om de constante weg te krijgen -> vector normalisatie
    # Normaliseren van x1 tot norm 1 zodanig dat we een eenheidsvector krijgen.
    x1 = x / np.linalg.norm(x)
    
    return x1

In [ ]:
#TEST
A = np.asmatrix([[-2, -2, 4],
                 [-4,  1, 2],
                 [ 2,  2, 5]])
eigval = 6
x0 = np.asmatrix([[1, 2, 3]]).transpose()
n = range(50)

errors = []
for i in n:
    x1 = rayleigh(A, x0, eigval, i)
    errors.append(np.linalg.norm(A*x1 - eigval*x1)) #A*x = eigval*x
plot_errors(n, errors)